In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

In [2]:
df = pd.read_csv("transfusion_5%.csv")

In [3]:
df

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2.0,50.0,12500.0,98.0,1.0
1,NaN,13.0,3250.0,NaN,1.0
2,1.0,16.0,4000.0,35.0,1.0
3,2.0,NaN,5000.0,45.0,1.0
4,1.0,24.0,6000.0,77.0,0.0
...,...,...,...,...,...
743,NaN,2.0,500.0,38.0,0.0
744,21.0,2.0,500.0,52.0,0.0
745,23.0,NaN,750.0,62.0,0.0
746,39.0,1.0,250.0,39.0,0.0


In [4]:
df.dtypes

Recency (months)                              float64
Frequency (times)                             float64
Monetary (c.c. blood)                         float64
Time (months)                                 float64
whether he/she donated blood in March 2007    float64
dtype: object

In [5]:
df.isnull().sum()

Recency (months)                              46
Frequency (times)                             46
Monetary (c.c. blood)                         55
Time (months)                                 40
whether he/she donated blood in March 2007     0
dtype: int64

In [6]:
null_counts = df.isnull().sum().sum()
total = df.size
null_percentages = 100 * null_counts / total
print("total nilai :",total)
print("nilai hilang :",null_counts)
print("persentase :",null_percentages)

total nilai : 3740
nilai hilang : 187
persentase : 5.0


In [7]:
null_counts = df.isnull().sum()
null_percentages = 100 * null_counts / len(df)
null_percentages

Recency (months)                              6.149733
Frequency (times)                             6.149733
Monetary (c.c. blood)                         7.352941
Time (months)                                 5.347594
whether he/she donated blood in March 2007    0.000000
dtype: float64

# Imputasi

In [8]:
def jarak(df2,col):
    euc_dist[col] = []
    euc_dist_append =euc_dist[col].append
    for row in range(df2.shape[0]):
        current = df2.index[df2.iloc[:,col] == 0].tolist()
        current = current[0]
        a = df2.iloc[current,col+1:].tolist()
        b = df2.iloc[current,:col].tolist()
        x1 = b+a
        if(row is not current):
            a=df2.iloc[row,col+1:].tolist()
            b=df2.iloc[row,:col].tolist()
            x2=b+a
        else:
            continue
        dist = euclidean(x1,x2)
        euc_dist_append([dist,df2.iloc[row,col]])
#         print(" x1 = ",x1,"\n x2 = ",x2,"\n distance = ",euc_dist[col])

    lst = list(euc_dist[col])
    lst.sort()
    euc_dist[col] = lst
#     print(euc_dist[col])

In [9]:
from collections import Counter
def hitung(val, k, temp=None, i=0, counter=0):
    counts = Counter(val)
    total_count = len(counts)
#     print("max_count:",max_count)
#     print("max_vals:",max_vals)
#     print("jumlah val:", total_count)
    if counter >= int((k/2)+1) and total_count == 1 :
        print("median : ",int((k/2)+1))
        return temp
    elif i is not k:
        if temp is None:
            temp = val[i]
        elif temp != val[i]:
            temp = val[i]
            counter = 0
        # print(f"temp = {temp} , counter = {counter} , i = {i} , k = {k} , median = {int((k/2)+1)}, val[i] = {val[i]}")
        return hitung(val,k,temp,i+1,counter+1)
    else:
        return None
    
def Final(df,col,k):
    terdekat = []
    mean = 0
    std = 0
    diffsquared = 0
    sum_diffsquared = 0
    fuzzy_val = []
    euc_val = []
    terdekat_append = terdekat.append
    fuzzy_append = fuzzy_val.append
    
    for val in range(k):
        terdekat_append(euc_dist[col][val])
    
    for x,y in terdekat:
        mean += y  
        euc_val.append(y)
    mean /= len(terdekat)
    
    print(euc_val)
    
    for x,y in terdekat:
        diffsquared = (y-mean)**2
        sum_diffsquared = diffsquared + sum_diffsquared
    std_dev = ((sum_diffsquared)/len(terdekat))**(1/2)
    print("mean : ",mean)
    print("standar deviasi : ",std_dev)
    
    euc_val.sort()
    fuzzy = hitung(euc_val,5)
    
    for x,y in terdekat:
        if fuzzy is None:
            fuzzy_append((y,gaussian(y,mean,std_dev)))
        else:
            fuzzy_val.append((y,fuzzy))
        
    print("fuzzy : ",fuzzy_val)
    print("fuzzy : ",fuzzy)
    
    pembilang = 0
    penyebut = 0 
    
    for nilai,bobot in fuzzy_val:
        pembilang += nilai*bobot
        penyebut += bobot
            
    print("pembilang : ", pembilang)
    print("penyebut : ",penyebut)
    
    if fuzzy is not None:
        tertimbang = fuzzy
    else:
        tertimbang = pembilang / penyebut
    
    print("rata-rata tertimbang : ",tertimbang)
    index = dummy.index[dummy.iloc[:,col] == 0].tolist()
    index = index[0]
    dummy.iloc[index,col] = tertimbang
    print(" ")

In [10]:
def gaussian(val, mean, std_dev):
    gaussian_mf = 2.718**((-1 * (val-mean)**2) / (2 * std_dev**2))
    return gaussian_mf

In [11]:
data = df.iloc[:, :-1]
df2 = data.copy()
dummy = df2.fillna(0)
dummy

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
0,2.0,50.0,12500.0,98.0
1,0.0,13.0,3250.0,0.0
2,1.0,16.0,4000.0,35.0
3,2.0,0.0,5000.0,45.0
4,1.0,24.0,6000.0,77.0
...,...,...,...,...
743,0.0,2.0,500.0,38.0
744,21.0,2.0,500.0,52.0
745,23.0,0.0,750.0,62.0
746,39.0,1.0,250.0,39.0


In [12]:
count_zeros = np.count_nonzero(dummy == 0)

print("Jumlah angka 0 dalam dataset:", count_zeros)

Jumlah angka 0 dalam dataset: 191


In [13]:
euc_dist = {}

In [14]:
for i in range(dummy.shape[1]):
    print("Kolom :",i+1)
    current = dummy.index[dummy.iloc[:,i] == 0].tolist()
    for x in current:
        jarak(dummy,i)
        Final(dummy,i,5)

Kolom : 1
[12.0, 2.0, 4.0, 1.0, 2.0]
mean :  4.2
standar deviasi :  4.019950248448357
fuzzy :  [(12.0, 0.15225012298132615), (2.0, 0.860934416927647), (4.0, 0.9987632699414724), (1.0, 0.7284772861908345), (2.0, 0.860934416927647)]
fuzzy :  None
pembilang :  9.994269509443226
penyebut :  3.601359512968927
rata-rata tertimbang :  2.7751379648304106
 
[2.0, 2.0, 2.0, 2.0, 4.0]
mean :  2.4
standar deviasi :  0.8
fuzzy :  [(2.0, 0.8825083402897206), (2.0, 0.8825083402897206), (2.0, 0.8825083402897206), (2.0, 0.8825083402897206), (4.0, 0.13536335042701722)]
fuzzy :  None
pembilang :  7.601520124025834
penyebut :  3.6653967115858994
rata-rata tertimbang :  2.0738601363389395
 
[23.0, 2.0, 8.0, 12.0, 7.0]
mean :  10.4
standar deviasi :  7.059745037889116
fuzzy :  [(23.0, 0.20341002862858829), (2.0, 0.49273105850907783), (8.0, 0.9438585923175061), (12.0, 0.9746473944159376), (7.0, 0.8905116594712178)]
fuzzy :  None
pembilang :  31.14411186330551
penyebut :  3.5051587333423275
rata-rata tertimba

[0.0, 14.0, 16.0, 14.0, 2.0]
mean :  9.2
standar deviasi :  6.764613810115105
fuzzy :  [(0.0, 0.39663878060921426), (14.0, 0.7774607259794593), (16.0, 0.6033894906414813), (14.0, 0.7774607259794593), (2.0, 0.5675792815671518)]
fuzzy :  None
pembilang :  32.55829074082286
penyebut :  3.1225290047767658
rata-rata tertimbang :  10.426897777735936
 
[0.0, 2.0, 16.0, 11.0, 4.0]
mean :  6.6
standar deviasi :  5.986651818838307
fuzzy :  [(0.0, 0.5446358439791947), (2.0, 0.7444048155736323), (16.0, 0.29154145192314207), (11.0, 0.7633327145743497), (4.0, 0.9100113041055821)]
fuzzy :  None
pembilang :  18.190177938657712
penyebut :  3.2539261301559006
rata-rata tertimbang :  5.59022461207077
 
[0.0, 11.0, 21.0, 21.0, 2.0]
mean :  11.0
standar deviasi :  8.966604708583958
fuzzy :  [(0.0, 0.471229736287579), (11.0, 1.0), (21.0, 0.5369630113594511), (21.0, 0.5369630113594511), (2.0, 0.604303264043517)]
fuzzy :  None
pembilang :  34.76105300518398
penyebut :  3.1494590230499986
rata-rata tertimbang 

[0.0, 11.0, 4.0, 14.0, 21.0]
mean :  10.0
standar deviasi :  7.402702209328699
fuzzy :  [(0.0, 0.40159379690122216), (11.0, 0.9909183480214933), (4.0, 0.720051100619894), (14.0, 0.8641833992355583), (21.0, 0.3315753716992485)]
fuzzy :  None
pembilang :  32.84195662569804
penyebut :  3.308322016477416
rata-rata tertimbang :  9.927073743766634
 
Kolom : 2
[20.0, 21.0, 21.0, 19.0, 19.0]
mean :  20.0
standar deviasi :  0.8944271909999159
fuzzy :  [(20.0, 1.0), (21.0, 0.5352961160122968), (21.0, 0.5352961160122968), (19.0, 0.5352961160122968), (19.0, 0.5352961160122968)]
fuzzy :  None
pembilang :  62.82368928098374
penyebut :  3.1411844640491875
rata-rata tertimbang :  19.999999999999996
 
[6.0, 6.0, 6.0, 6.0, 6.0]
mean :  6.0
standar deviasi :  0.0
median :  3
fuzzy :  [(6.0, 6.0), (6.0, 6.0), (6.0, 6.0), (6.0, 6.0), (6.0, 6.0)]
fuzzy :  6.0
pembilang :  180.0
penyebut :  30.0
rata-rata tertimbang :  6.0
 
[8.0, 8.0, 8.0, 8.0, 8.0]
mean :  8.0
standar deviasi :  0.0
median :  3
fuzzy :  [(

[0.0, 1.8319229625506361, 6.0, 0.0, 6.0]
mean :  2.766384592510127
standar deviasi :  2.7236564824637184
fuzzy :  [(0.0, 0.5970482508494107), (1.8319229625506361, 0.942848596089894), (6.0, 0.4942626289116096), (0.0, 0.5970482508494107), (6.0, 0.4942626289116096)]
fuzzy :  None
pembilang :  7.658377540325022
penyebut :  3.125470355611934
rata-rata tertimbang :  2.4503120071428715
 
[0.0, 2.0, 2.0, 2.0, 2.0]
mean :  1.6
standar deviasi :  0.8000000000000002
fuzzy :  [(0.0, 0.13536335042701728), (2.0, 0.8825083402897206), (2.0, 0.8825083402897206), (2.0, 0.8825083402897206), (2.0, 0.8825083402897206)]
fuzzy :  None
pembilang :  7.060066722317765
penyebut :  3.6653967115858994
rata-rata tertimbang :  1.9261398636610607
 
[0.0, 0.0, 4.0, 4.0, 4.0]
mean :  2.4
standar deviasi :  1.9595917942265426
fuzzy :  [(0.0, 0.4724032869055458), (0.0, 0.4724032869055458), (4.0, 0.7165560753439463), (4.0, 0.7165560753439463), (4.0, 0.7165560753439463)]
fuzzy :  None
pembilang :  8.598672904127355
penyebu

[5750.0, 4750.0, 4000.0, 4250.0, 4000.0]
mean :  4550.0
standar deviasi :  659.545297913646
fuzzy :  [(5750.0, 0.1910919031284711), (4750.0, 0.955068470320002), (4000.0, 0.70633538954109), (4250.0, 0.9017323342552541), (4000.0, 0.70633538954109)]
fuzzy :  None
pembilang :  15118.399213922268
penyebut :  3.460563486785907
rata-rata tertimbang :  4368.7680551596795
 
[1500.0, 1500.0, 1250.0, 1500.0, 2000.0]
mean :  1550.0
standar deviasi :  244.94897427831782
fuzzy :  [(1500.0, 0.9793842968861823), (1500.0, 0.9793842968861823), (1250.0, 0.4724032869055457), (1500.0, 0.9793842968861823), (2000.0, 0.18501376841789724)]
fuzzy :  None
pembilang :  5367.760981455547
penyebut :  3.5955699459819903
rata-rata tertimbang :  1492.8818134810479
 
[500.0, 1000.0, 500.0, 500.0, 750.0]
mean :  650.0
standar deviasi :  200.0
fuzzy :  [(500.0, 0.7548616143394907), (1000.0, 0.21629950522797875), (500.0, 0.7548616143394907), (500.0, 0.7548616143394907), (750.0, 0.8825083402897205)]
fuzzy :  None
pembilang

[0.0, 250.0, 250.0, 250.0, 250.0]
mean :  200.0
standar deviasi :  100.0
fuzzy :  [(0.0, 0.13536335042701722), (250.0, 0.8825083402897205), (250.0, 0.8825083402897205), (250.0, 0.8825083402897205), (250.0, 0.8825083402897205)]
fuzzy :  None
pembilang :  882.5083402897204
penyebut :  3.665396711585899
rata-rata tertimbang :  240.76748295763258
 
[0.0, 2000.0, 2250.0, 1250.0, 1750.0]
mean :  1450.0
standar deviasi :  796.8688725254614
fuzzy :  [(0.0, 0.19102706479278553), (2000.0, 0.7880732158481539), (2250.0, 0.6041790059233539), (1250.0, 0.9689979357856628), (1750.0, 0.9315934293961228)]
fuzzy :  None
pembilang :  5777.085116199148
penyebut :  3.483870651746079
rata-rata tertimbang :  1658.237544870885
 
[0.0, 500.0, 500.0, 500.0, 750.0]
mean :  450.0
standar deviasi :  244.94897427831782
fuzzy :  [(0.0, 0.18501376841789724), (500.0, 0.9793842968861823), (500.0, 0.9793842968861823), (500.0, 0.9793842968861823), (750.0, 0.4724032869055457)]
fuzzy :  None
pembilang :  1823.378910508433
p

[0.0, 1000.0, 1250.0, 966.2035568335373, 1250.0]
mean :  893.2407113667075
standar deviasi :  462.4185270992655
fuzzy :  [(0.0, 0.15482125201255906), (1000.0, 0.9737038507075584), (1250.0, 0.742612480009144), (966.2035568335373, 0.9876303315632499), (1250.0, 0.742612480009144)]
fuzzy :  None
pembilang :  3784.4869899235164
penyebut :  3.601380394301655
rata-rata tertimbang :  1050.8434476712275
 
[0.0, 750.0, 750.0, 750.0, 750.0]
mean :  600.0
standar deviasi :  300.0
fuzzy :  [(0.0, 0.13536335042701722), (750.0, 0.8825083402897205), (750.0, 0.8825083402897205), (750.0, 0.8825083402897205), (750.0, 0.8825083402897205)]
fuzzy :  None
pembilang :  2647.5250208691614
penyebut :  3.665396711585899
rata-rata tertimbang :  722.3024488728978
 
[0.0, 1250.0, 1500.0, 2000.0, 2000.0]
mean :  1350.0
standar deviasi :  734.8469228349534
fuzzy :  [(0.0, 0.18501376841789713), (1250.0, 0.990784426873854), (1500.0, 0.9793842968861823), (2000.0, 0.6762698190061995), (2000.0, 0.6762698190061995)]
fuzzy 

[0.0, 0.0, 18.113738130314946, 22.0, 28.0]
mean :  13.622747626062989
standar deviasi :  11.560374082499248
fuzzy :  [(0.0, 0.49945310121702385), (0.0, 0.49945310121702385), (18.113738130314946, 0.9273250925329528), (22.0, 0.769100753795808), (28.0, 0.46149998607032233)]
fuzzy :  None
pembilang :  46.63954008128879
penyebut :  3.1568320348331307
rata-rata tertimbang :  14.77415952659456
 
[0.0, 14.0, 14.0, 14.0, 14.0]
mean :  11.2
standar deviasi :  5.6000000000000005
fuzzy :  [(0.0, 0.13536335042701733), (14.0, 0.8825083402897204), (14.0, 0.8825083402897204), (14.0, 0.8825083402897204), (14.0, 0.8825083402897204)]
fuzzy :  None
pembilang :  49.42046705622434
penyebut :  3.665396711585899
rata-rata tertimbang :  13.482979045627424
 
[0.0, 0.0, 2.0, 2.0, 2.511424576967142]
mean :  1.3022849153934284
standar deviasi :  1.0795854205655284
fuzzy :  [(0.0, 0.483123384313592), (0.0, 0.483123384313592), (2.0, 0.8115432180850728), (2.0, 0.8115432180850728), (2.511424576967142, 0.53411766987255

In [15]:
dummy

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
0,2.000000,50.00000,12500.0,98.000000
1,2.775138,13.00000,3250.0,46.405865
2,1.000000,16.00000,4000.0,35.000000
3,2.000000,20.00000,5000.0,45.000000
4,1.000000,24.00000,6000.0,77.000000
...,...,...,...,...
743,9.927074,2.00000,500.0,38.000000
744,21.000000,2.00000,500.0,52.000000
745,23.000000,2.88921,750.0,62.000000
746,39.000000,1.00000,250.0,39.000000


In [16]:
dummy.describe()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
count,748.000000,748.000000,748.000000,748.000000
mean,9.422051,5.495783,1362.485104,33.725853
std,7.973538,5.842610,1454.335895,24.019438
min,0.926492,0.953771,173.669877,1.468750
25%,3.000000,2.000000,500.000000,15.795478
50%,7.263879,4.000000,1000.000000,28.000000
75%,14.000000,7.000000,1750.000000,48.467407
max,74.000000,50.000000,12500.000000,98.000000


# Klasifikasi

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [29]:
x = dummy.values
y = df.iloc[:, -1].values

In [30]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=72)

In [31]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [32]:
classifier = DecisionTreeClassifier()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[123  23]
 [ 26  15]]
0.7379679144385026
